## Collaboration network

In [1]:
import requests

In [2]:
# obtain author id to start with
base_url_authors = 'https://api.openalex.org/authors'

name = 'joeri both'

params = {'search': name}
r = requests.get(base_url_authors, params)
data = r.json()
author_id = data['results'][0]['id']

In [5]:
def colab_net(author_id, level=2, nodes=[], nodes_n=[], edges=[]):

    # level: depth of the network, aka "collaborators of collaborators"
    
    base_url_works = 'https://api.openalex.org/works'
    
    params = {'filter': f'author.id:{author_id}'}
    r = requests.get(base_url_works, params)
    data = r.json()

    author_ids = set()
    for work in data['results']:
        authors = []
        institutes = []
        for authorship in work['authorships']:
            author_ids.add(authorship['author'].get('id'))
            name = authorship['author'].get('display_name')
            if name:
                authors.append(name)
                for institute in authorship['institutions']:
                    iname = institute.get('display_name')
                    if iname:
                        institutes.append(iname)
                        edges.append({'n1': name, 'n2': iname, 'type': 'works_at'})

        for n in authors: 
            if n not in nodes_n:
                nodes.append({'n': n, 'entity': 'author'})  # nodes have to be unique
                nodes_n.append(n)
        [edges.append({'n1': n1, 'n2': n2, 'type': 'works_with'}) for n1 in authors for n2 in authors if n1 != n2]
        
        for n in institutes: 
            if n not in nodes_n:
                nodes.append({'n': n, 'entity': 'institute'})
                nodes_n.append(n)

    if level > 1:
        for aid in author_ids:
            colab_net(aid, level-1, nodes=nodes, nodes_n=nodes_n, edges=edges)

    return nodes, edges

In [6]:
nodes, edges = colab_net(author_id)

In [7]:
print(len(nodes))
print(len(edges))

1730
64762


In [8]:
from ipysigma import Sigma

In [9]:
import pandas as pd
from pelote import edges_table_to_graph, tables_to_graph

In [10]:
#df = pd.DataFrame(edges)
#g = edges_table_to_graph(
#    df,
#    edge_source_col="n1",
#    edge_target_col="n2",
#    edge_data=['type']
#)

g = tables_to_graph(
    nodes, 
    edges, 
    node_col="n", node_data=["entity"], 
    edge_data=["type"], 
    edge_source_col="n1",
    edge_target_col="n2",
    directed=False
)

Sigma(g, node_color='entity', node_size=g.degree)

Sigma(nx.Graph with 1,730 nodes and 27,029 edges)